In [ ]:
import pandas as pd

1. caricare i file in un opportuno dataframe

In [ ]:
parcheggi = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/sosta_turistici.csv", sep = ";")

2. visualizzare il numero di parcheggi di cui non sono note le regole

In [ ]:
len(parcheggi[parcheggi.regole.isnull()])

3. eliminare i parcheggi di cui non sono note le regole

In [ ]:
parcheggi = parcheggi.drop(parcheggi[parcheggi.regole.isnull()].index)

4. fare in modo che il campo "costo_ora" contenga il prezzo sottoforma di valori interi (se presenti caratteri particolari, come ad esempio -, inserire il valore 0)

In [ ]:
parcheggi['costo_ora'] = parcheggi['costo_ora'].str.replace('-', '0')
parcheggi['costo_ora'] = parcheggi['costo_ora'].str.replace('€', '')
parcheggi['costo_ora'] = parcheggi['costo_ora'].astype(int)

5. aggiungere al dataframe due colonne chiamate 'oraInizio' e 'oraFine' che contengono l'ora di quando si può iniziare a parcheggiare e quella in cui si deve terminare il parcheggio. Prendere i dati dalla colonna regole.

In [ ]:
parcheggi['oraInizio'] = parcheggi['regole'].str[0]
parcheggi['oraFine'] = parcheggi['regole'].str[2:4]

6. calcolare quanto raccoglierebbe il comune di Milano se tutti i parcheggi a pagamento venissero occupati per tutto il tempo possibile e per tutti gli stalli disponibili

In [ ]:
parcheggi['oraFine'] = parcheggi['oraFine'].astype(int)
parcheggi['oraInizio'] = parcheggi['oraInizio'].astype(int)
parcheggi["totale"] = parcheggi['costo_ora'] * (parcheggi['oraFine'] - parcheggi['oraInizio'])
prezzoTotale = parcheggi['totale'].sum()
print(prezzoTotale)

7. trovare i parcheggi che sono vicini ad una metropolitana (hanno la sigla della metropolitana nel campo 'localizz')

In [ ]:
metropolitane = ["M1", "M2", "M3", "M5"]
parcheggi[parcheggi['localizzaz'].str.contains('|'.join(metropolitane))]

8. trovare i parcheggi su strada che permettono una sosta lunga e che hanno un numero di stalli compreso tra 7 e 10

In [ ]:
parcheggi[(parcheggi['tipologia'] == "su strada") & (parcheggi['tipo_sosta'] == "LUNGA") & (parcheggi['num_stalli'] >= 7) & (parcheggi['num_stalli'] <= 10)]

9. calcolare il numero di stalli per ogni tipologia

In [ ]:
dfStalli = parcheggi.groupby('tipologia')[['num_stalli']].sum().reset_index()
dfStalli

10. trovare la tipologia che ha più stalli

In [ ]:
dfStalli[dfStalli['num_stalli'] == dfStalli['num_stalli'].max()][['tipologia']]

11. calcolare il numero di stalli per ogni tipologia e per ogni provenienze. Chiamare il risultato dfTotale

In [ ]:
dfTotale = parcheggi.groupby(['tipologia', 'provenienza'])[['num_stalli']].sum().reset_index()
dfTotale

12. visualizzare le tipologie che non hanno le due provenienze (ad esempio 'area_chiusa' ha solo la provenienza fuori)

13. aggiungere a dfTotale due righe, una per tipologia area chiusa e provenienza dentro con valori uguali a 0 e l'altra con tipologia area parcheggio e provenienza dentro con valori uguali a 0

In [ ]:
dfTotale.loc[4] = ['area chiusa', 'dentro', 0]
dfTotale.loc[5] = ['area parcheggio', 'dentro', 0]

14. usare i dati dell'esercizio precedente per creare un grafico a barre verticali con sulle ascisse le tipologie e sulle ordinate i valori della provenienza, una barra per il valore dentro e una per il valore fuori

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
labels = list(dfTotale['tipologia'].unique())  #unique: rimuove i duplicati
fuori = dfTotale[dfTotale['provenienza'] == "fuori"]
dentro = dfTotale[dfTotale['provenienza'] == "dentro"]

finale = fuori.merge(dentro, on = "tipologia")

x = np.arange(len(labels))
width = 0.35

fig, ax1 = plt.subplots(figsize=(15,8))
ax1.bar(x - width/2, finale["num_stalli_x"], width, label='fuori')
ax1.bar(x + width/2, finale["num_stalli_y"], width, label='dentro')


ax1.set_ylabel('num_stalli')
ax1.set_xlabel('tipologia')
ax1.set_title('numero di stalli per ogni tipologia e provenienza')
default_x_ticks = range(len(labels))
plt.xticks(default_x_ticks, labels)
ax1.legend()

fig.tight_layout()
plt.show()

15. fare un grafico analogo a quello sopra, mettendo non due colonne distinte, ma un dato sopra l'altro in modo da formare dei 'blocchi'

In [ ]:
labels = list(dfTotale['tipologia'].unique())
fuori = dfTotale[dfTotale['provenienza'] == "fuori"]
dentro = dfTotale[dfTotale['provenienza'] == "dentro"]

finale = fuori.merge(dentro, on = "tipologia")

x = np.arange(len(labels))
width = 0.35


fig, ax2 = plt.subplots(figsize=(15,8))
ax2.bar(x, finale["num_stalli_x"]  + finale["num_stalli_y"] , width, label='fuori')
ax2.bar(x, finale["num_stalli_y"], width, label='dentro')


ax2.set_ylabel('num_stalli')
ax2.set_xlabel('tipologia')
ax2.set_title('numero di stalli per ogni tipologia e provenienza')
default_x_ticks = range(len(labels))
plt.xticks(default_x_ticks, labels)
ax2.legend()

fig.tight_layout()
plt.show()

16. rappresentare i due grafici in un unico grafico e salvarli su un file pdf

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 10))

ax1.bar(x - width/2, finale["num_stalli_x"], width, label='fuori')
ax1.bar(x + width/2, finale["num_stalli_y"], width, label='dentro')


ax1.set_ylabel('num_stalli')
ax1.set_xlabel('tipologia')
ax1.set_title('numero di stalli per ogni tipologia e provenienza')
default_x_ticks = range(len(labels))
plt.xticks(default_x_ticks, labels)
ax1.legend()



ax2.bar(x, finale["num_stalli_x"]  + finale["num_stalli_y"] , width, label='fuori')
ax2.bar(x, finale["num_stalli_y"], width, label='dentro')


ax2.set_ylabel('num_stalli')
ax2.set_xlabel('tipologia')
ax2.set_title('numero di stalli per ogni tipologia e provenienza')
default_x_ticks = range(len(labels))
plt.xticks(default_x_ticks, labels)
ax2.legend()

plt.tight_layout()
plt.savefig("grafici.pdf", format="pdf", bbox_inches="tight")   #salvare il grafico in un file pdf
plt.show

17. creare una funzione percentuale(parte, totale) che restituisca la percentuale del parametro parte rispetto al totale

In [ ]:
def percentuale(parte, totale):
  return (parte / totale) * 100 

print(percentuale(2, 8))

18. utilizzare questa funzione per aggiungere al dataset dfTotale una colonna percentuale contenente la percentuale del numero di stalli rispetto al totale degli stalli



In [ ]:
dfTotale['percentuale'] = percentuale(dfTotale['num_stalli'], parcheggi['num_stalli'].sum())